In [1]:
import re
import os
from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#from sklearn.neighbors import NearestNeighbors
#from sklearn.cluster import KMeans
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor

In [2]:
predic = pd.read_csv('kor_hmm_pred.csv', low_memory=False)
indexing_predic = predic.set_index('date') 

predic_usa = pd.read_csv('usa_hmm_pred.csv', low_memory=False)
indexing_predic_usa = predic_usa.set_index('date') 

bt = pd.read_csv('data_d_kor.csv')
NaN_processing_bt = bt.fillna(0) 
indexing_bt = NaN_processing_bt.set_index('date') 

btusa = pd.read_csv('data_d_usa.csv')
NaN_processing_btusa = btusa.fillna(0) 
indexing_bt_usa = NaN_processing_btusa.set_index('date') 





In [3]:
class MDD:

    def __init__(self):
        while (1):
            try:
                print("\n MDD는 2008년과 2020을 테스트 할 수 있습니다. \n")
                
                self.M = input(" 년도를 입력하세요 형식 : xxxx,  2008 or 2020 ")
                
                
                if self.M =='2008':
                    self.S = '2007-10'
                    self.E = '2009-04'               
                elif self.M == '2020':
                    self.S = '2020-01'
                    self.E = '2020-04'
                    
                    
                else:
                    print('다시 입력하세요')
                    continue
               
            except:

                self.Q = input("혹시 프로그램을 종료하고 싶다면 q를 입력해주세요.(대소문자 모두 가능) : ")
                print('\n\n\n')
                if self.Q == 'q' or self.Q == 'Q':
                    print('\n프로그램을 종료합니다. 감사합니다.')
                    exit()
                else:
                    continue
            

            self.predic12_1 = indexing_predic.loc[:, 'pred']
            self.predic12_1_set_SE = indexing_predic.loc[self.S : self.E]
              

            self.predic12_1_usa = indexing_predic_usa.loc[:, 'pred']
            self.predic12_1_set_SE_usa = indexing_predic_usa.loc[self.S : self.E]
     

            if self.predic12_1_set_SE.empty:
                print('\n---------------------------------------------------')
                print(" 날짜입력 오류입니다. 1월인경우 01로 입력 바랍니다.")
                print('---------------------------------------------------\n')

                continue


            else:
                break
                
        '''         
        while (1):
            print('미국과 한국의 포트폴리오 비율을 입력합니다. 입력값의 합이 10이 되게 해주세요 \n한국:미국 = 6:4 라면, 6입력  \n한국:미국 = 6:4 라면, 4입력')
            try:
                self.K = int(input("\n한국비율 입력. \n한국:미국 = 6:4 라면, 6입력 "))
                self.U = int(input("\n한국비율 입력. \n한국:미국 = 6:4 라면, 4입력 "))
      
                if (self.K + self.U) != 10:
                    print('\n한국 + 미국의 입력값이 총 10이 되게 해주세요\n')
                    continue
                else:
                    print('\n--------------------')
                    print("한국과 미국의 비율은 {} : {}".format(self.K, self.U))
                    print('--------------------\n')
                    break
            except:
                print('\n--------------------')
                print("숫자를 다시 입력해주세요")
                print('--------------------\n')
                continue

        '''    

        self.Collected_Data_fix_kor = pd.DataFrame()
        self.Collected_Data_fix_usa = pd.DataFrame()
        self.Collected_Data_fix_kor = indexing_bt.loc['{}'.format(self.S):'{}'.format(self.E)]
        self.Collected_Data_fix_usa = indexing_bt_usa.loc['{}'.format(self.S):'{}'.format(self.E)]

          
        
        
    # 고정된 포트폴리오를 계산하기 위한 함수 
    def Backtesting_manualset_kor(self, a, b, c, d, Data):

        stock = a
        bond = b
        gold = c
        commodity = d

        self.list_for_percent = []
        list_for_var = []
        list_for_mdd = []

        R = None

        for i in range(0, len(Data)):
            gold = gold * (1 + Data['gold'][i])  
            bond = bond * (1 + Data['k10bond'][i])
            stock = stock * (1 + Data['kospi'][i])
            commodity = commodity * (1 + Data['commodity'][i])

            sum_for_var = gold + bond + stock + commodity
            self.list_for_percent.append(sum_for_var - 100)
            list_for_mdd.append(sum_for_var)
            
            if R:  
                r = (sum_for_var - R) / R * 100
                list_for_var.append(r)
            R = sum_for_var
        
        mdd = (min(list_for_mdd)-max(list_for_mdd))/max(list_for_mdd)
        rate_of_return = (gold + bond + stock + commodity - 100)  
        var_portfolio = np.array(list_for_var).var()  
        Volatility_standard_deviation = np.sqrt(var_portfolio)
        Sharpe_ratio = (rate_of_return / 100) / Volatility_standard_deviation  
        #        mdd = max()
        
        print(
            "한국주식 {}%, 한국채권 10년 {}%, 금 {}%, 원자재 {}% 포트폴리오 의 결과 ".format(
                a, b, c, d))

        print('MDD %: {}\n'.format(round(mdd*100,4)))

        return self.list_for_percent


    # 전략 실행을 위한 백테스팅 함수
    def Backtesting_monthly_kor(self, a, b, c, d, Data, pre_sum, for_var):

        stock = pre_sum * a / 100
        bond = pre_sum * b / 100
        gold = pre_sum * c / 100
        commodity = pre_sum * d / 100

        list_for_mdd = []

        R = pre_sum
        for i in range(0, len(Data)):
            gold = gold * (1 + Data['gold'][i]) 
            bond = bond * (1 + Data['k10bond'][i])
            stock = stock * (1 + Data['kospi'][i])
            commodity = commodity * (1 + Data['commodity'][i])

            sum_for_mdd_var = gold + bond + stock + commodity
            list_for_mdd.append(sum_for_mdd_var)

            r = (sum_for_mdd_var - R) / R * 100
            for_var.append(r)

            R = sum_for_mdd_var

        self.df = pd.DataFrame(list_for_mdd)

        return self.df
    
    # 클래스 실행에 필요한 구간 데이터 모으는 함수
    def Select_Date_for_STG_kor(self, Start, End):  

        self.total = indexing_bt.loc['{}'.format(Start):'{}'.format(End)]

        return self.total
    def Select_Date_for_STG_usa(self, Start, End):  

        self.total = indexing_bt_usa.loc['{}'.format(Start):'{}'.format(End)]

        return self.total

    def Backtesting_manualset_usa(self, a, b, c, d, Data):

        stock = a
        bond = b
        gold = c
        commodity = d

        self.list_for_percent = []
        list_for_var = []
        list_for_mdd = []

        R = None

        for i in range(0, len(Data)):
            gold = gold * (1 + Data['gold'][i])  
            bond = bond * (1 + Data['u10bond'][i])
            stock = stock * (1 + Data['snp500'][i])
            commodity = commodity * (1 + Data['commodity'][i])

            sum_for_var = gold + bond + stock + commodity
            self.list_for_percent.append(sum_for_var - 100)
            list_for_mdd.append(sum_for_var)
            if R:  
                r = (sum_for_var - R) / R * 100
                list_for_var.append(r)
            R = sum_for_var
            
        mdd = (min(list_for_mdd)-max(list_for_mdd))/max(list_for_mdd)
        rate_of_return = (gold + bond + stock + commodity - 100)  
        var_portfolio = np.array(list_for_var).var()  
        Volatility_standard_deviation = np.sqrt(var_portfolio)
        Sharpe_ratio = (rate_of_return / 100) / Volatility_standard_deviation  
        return_dataframe = pd.DataFrame(
            {'Return %': [round(rate_of_return, 4)], 'Sharpe': [round(Sharpe_ratio, 4)],
             'Volatility': [round(Volatility_standard_deviation, 4)]})

        print(
            "미국주식 {}%, 미국채권 10년 {}%, 금 {}%, 원자재 {}% 포트폴리오 의 결과 ".format(
                a, b, c, d))
        
        print('MDD %: {}\n'.format(round(mdd*100,4)))

        return self.list_for_percent


    # 전략 실행을 위한 백테스팅 함수
    def Backtesting_monthly_usa(self, a, b, c, d, Data, pre_sum, for_var):

        stock = pre_sum * a / 100
        bond = pre_sum * b / 100
        gold = pre_sum * c / 100
        commodity = pre_sum * d / 100

        list_for_mdd = []

        R = pre_sum
        for i in range(0, len(Data)):
            gold = gold * (1 + Data['gold'][i]) 
            bond = bond * (1 + Data['u10bond'][i])
            stock = stock * (1 + Data['snp500'][i])
            commodity = commodity * (1 + Data['commodity'][i])

            sum_for_mdd_var = gold + bond + stock + commodity
            list_for_mdd.append(sum_for_mdd_var)

            r = (sum_for_mdd_var - R) / R * 100
            for_var.append(r)

            R = sum_for_mdd_var

        self.df = pd.DataFrame(list_for_mdd)

        return self.df    
        
        
    
        
        
    # 각 포트폴리오들의 백테스팅 결과값과 그리프를 프린트합니다.        
    def Run(self):

        print('\n\n')
        kor_6_4 = pd.DataFrame(self.Backtesting_manualset_kor(60, 40, 0, 0, self.Collected_Data_fix_kor),
                                                columns=['6_4'])
        usa_6_4 = pd.DataFrame(self.Backtesting_manualset_usa(60, 40, 0, 0, self.Collected_Data_fix_usa),
                                                columns=['6_4'])
        
        
        All_kor_stock = pd.DataFrame(self.Backtesting_manualset_kor(100, 0, 0, 0, self.Collected_Data_fix_kor),
                                                 columns=['10_0'])
        
        All_usa_stock = pd.DataFrame(self.Backtesting_manualset_usa(100, 0, 0, 0, self.Collected_Data_fix_usa),
                                                 columns=['10_0'])

        #print('아래는 Reference인 All Weather Portfolio 입니다.')
        kor_Allweather_reference = pd.DataFrame(
            self.Backtesting_manualset_kor(30, 50, 10, 10, self.Collected_Data_fix_kor), columns=['reference '])
        usa_Allweather_reference = pd.DataFrame(
            self.Backtesting_manualset_usa(30, 50, 10, 10, self.Collected_Data_fix_usa), columns=['reference '])

        start = [100]
  
        array_for_STG_kor = pd.DataFrame(start)
        array_for_STG_usa = pd.DataFrame(start)
        array_for_STG_kor_var = []
        array_for_STG_usa_var = []
  


        for i in self.predic12_1_set_SE.index:

   
            if i == self.predic12_1_set_SE.index[-1]:
                break

            else:
                if self.predic12_1.loc[i] == 0:


                    Colected_Data_Monthly_kor = self.Select_Date_for_STG_kor(i, ''.join(
                        self.predic12_1.index[(predic.index[predic.index[predic['date'] == i]] + 1).tolist()]))

                    array_for_STG_kor = pd.concat(
                        [array_for_STG_kor,
                         self.Backtesting_monthly_kor(10, 30, 50, 10, Colected_Data_Monthly_kor,
                                                         array_for_STG_kor.iloc[-1], array_for_STG_kor_var)],
                        ignore_index=True)




                elif self.predic12_1.loc[i] == 1:
  
                    Colected_Data_Monthly_kor = self.Select_Date_for_STG_kor(i, ''.join(
                        self.predic12_1.index[(predic.index[predic.index[predic['date'] == i]] + 1).tolist()]))
                    array_for_STG_kor = pd.concat(
                        [array_for_STG_kor,
                         self.Backtesting_monthly_kor(30, 50, 10, 10, Colected_Data_Monthly_kor,
                                                         array_for_STG_kor.iloc[-1], array_for_STG_kor_var)],
                        ignore_index=True)

                
                elif self.predic12_1.loc[i] == 2:

                    if (predic.index[predic.index[predic['date'] == i]] + 1).tolist()[0] < len(indexing_predic.index):
                        Colected_Data_Monthly_kor = self.Select_Date_for_STG_kor(i, ''.join(
                            self.predic12_1.index[(predic.index[predic.index[predic['date'] == i]] + 1).tolist()]))
                        array_for_STG_kor = pd.concat(
                            [array_for_STG_kor,
                             self.Backtesting_monthly_kor(24, 50, 16, 10, Colected_Data_Monthly_kor,
                                                             array_for_STG_kor.iloc[-1],
                                                             array_for_STG_kor_var)],
                            ignore_index=True)



                    else:
                        array_for_STG_kor = array_for_STG_kor.drop(index=0, axis=0)

                        continue

        
                elif self.predic12_1.loc[i] == 3:

                    Colected_Data_Monthly_kor = self.Select_Date_for_STG_kor(i, ''.join(
                        self.predic12_1.index[(predic.index[predic.index[predic['date'] == i]] + 1).tolist()]))
                    array_for_STG_kor = pd.concat(
                        [array_for_STG_kor,
                         self.Backtesting_monthly_kor(10, 30, 10, 50, Colected_Data_Monthly_kor,
                                                         array_for_STG_kor.iloc[-1], array_for_STG_kor_var)],
                        ignore_index=True)



                else:
                    print('Something.. Error')
                    
  

        for i in self.predic12_1_set_SE_usa.index:
    
    
            if i == self.predic12_1_set_SE_usa.index[-1]:
                break
    
            else:
                if self.predic12_1_usa.loc[i] == 0:
    
                    Colected_Data_Monthly_usa = self.Select_Date_for_STG_usa(i, ''.join(
                        self.predic12_1_usa.index[(predic_usa.index[predic_usa.index[predic_usa['date'] == i]] + 1).tolist()]))
    
                    array_for_STG_usa = pd.concat(
                        [array_for_STG_usa,
                         self.Backtesting_monthly_usa(10, 50, 30, 10, Colected_Data_Monthly_usa,
                                                      array_for_STG_usa.iloc[-1], array_for_STG_usa_var)],
                        ignore_index=True)
    
    
    
    
                elif self.predic12_1_usa.loc[i] == 1:
    
                    Colected_Data_Monthly_usa = self.Select_Date_for_STG_usa(i, ''.join(
                        self.predic12_1_usa.index[(predic_usa.index[predic_usa.index[predic_usa['date'] == i]] + 1).tolist()]))
                    array_for_STG_usa = pd.concat(
                        [array_for_STG_usa,
                         self.Backtesting_monthly_usa(12, 50, 28, 10, Colected_Data_Monthly_usa,
                                                      array_for_STG_usa.iloc[-1], array_for_STG_usa_var)],
                        ignore_index=True)
    
    
                elif self.predic12_1_usa.loc[i] == 2:
    
                    if (predic_usa.index[predic_usa.index[predic_usa['date'] == i]] + 1).tolist()[0] < len(indexing_predic_usa.index):
                        Colected_Data_Monthly_usa = self.Select_Date_for_STG_usa(i, ''.join(
                            self.predic12_1_usa.index[(predic_usa.index[predic_usa.index[predic_usa['date'] == i]] + 1).tolist()]))
                        array_for_STG_usa = pd.concat(
                            [array_for_STG_usa,
                             self.Backtesting_monthly_usa(30, 50, 10, 10, Colected_Data_Monthly_usa,
                                                          array_for_STG_usa.iloc[-1],
                                                          array_for_STG_usa_var)],
                            ignore_index=True)
    
    
    
                    else:
                        array_for_STG_usa = array_for_STG_usa.drop(index=0, axis=0)
    
                        continue
    
    
                elif self.predic12_1_usa.loc[i] == 3:
    
                    Colected_Data_Monthly_usa = self.Select_Date_for_STG_usa(i, ''.join(
                        self.predic12_1_usa.index[(predic_usa.index[predic_usa.index[predic_usa['date'] == i]] + 1).tolist()]))
                    array_for_STG_usa = pd.concat(
                        [array_for_STG_usa,
                         self.Backtesting_monthly_usa(10, 50, 30, 10, Colected_Data_Monthly_usa,
                                                      array_for_STG_usa.iloc[-1], array_for_STG_usa_var)],
                        ignore_index=True)
    
    
    
                else:
                    print('Something.. Error')
            
    
       

        kor_var = np.array(array_for_STG_kor_var).var()
        kor_standard_deviation = np.sqrt(kor_var)
        array_for_STG_kor = array_for_STG_kor.drop([0])
        

        
        usa_var = np.array(array_for_STG_usa_var).var()
        usa_standard_deviation = np.sqrt(usa_var)
        array_for_STG_usa = array_for_STG_usa.drop([0])
        
        
        
        KORmdd = (((array_for_STG_kor.min())-(array_for_STG_kor.max()))*100/(array_for_STG_kor.max())) 
        USAmdd = (((array_for_STG_usa.min())-(array_for_STG_usa.max()))*100/(array_for_STG_usa.max()))
        
        print("전략 kor 의 결과 ")
        print('MDD %: {}\n'.format( round(KORmdd.iloc[0],4)  ))

        print("전략 usa  의 결과 ")
        print('MDD %: {}\n'.format( round(USAmdd.iloc[0],4)  ))

        x = All_kor_stock.index  
        y = All_kor_stock.to_numpy()
        y1 = All_usa_stock.to_numpy()
        #y2 = Fixed_Allweather10_0.to_numpy()
        #y10 = Fixed_Allweather_reference.to_numpy()
        
        
    


        y4 = array_for_STG_kor.to_numpy() - 100
        y5 = array_for_STG_usa.to_numpy() - 100
  
        y111 = kor_6_4.to_numpy()
        y112 = usa_6_4.to_numpy()
        y113 = kor_Allweather_reference.to_numpy()
        y114 = usa_Allweather_reference.to_numpy()        
        y115 = y113*0.5 + y114*0.5 
   


        y10 = y4*0.5 + y5*0.5



        plt.figure(figsize=(40, 18))
        plt.plot(x, y, c='blue', alpha=0.2, label='KOR Stock 100%, Portfolio')
        plt.plot(x, y1, c='orange', alpha=0.2, label='USA Stock 100%, Portfolio')
        #plt.plot(x, y2, c='green', alpha=0.2, label='Stock 100% Portfolio')
        #plt.plot(x, y10, c='black', alpha=0.6, label='Ray Dalio Reference AWP')


        plt.plot(x, y4, c='purple', label='Our strategy_kor')
        plt.plot(x, y5, c='red', label='Our strategy_usa')

        plt.plot(x, y111, alpha=0.6, label='KOR Stock 60%, Bond 40% Portfolio')        
        plt.plot(x, y112, alpha=0.6,label='USA Stock 60%, Bond 40% Portfolio')
        plt.plot(x, y113, alpha=0.6,label='KOR Ray Dalio Reference AWP')        
        plt.plot(x, y114, alpha=0.6,label='USA Ray Dalio Reference AWP')        
        plt.plot(x, y115, alpha=0.6,label='KOR : USA = 1:1 Ray Dalio Reference AWP')         
        
        plt.plot(x, y10,alpha=0.4,  label='Our strategy_KOR:USA = 5:5')

        plt.legend(loc='upper left', frameon=False, fontsize=15)
        plt.axhline(y=0, color='black', linewidth=1, alpha=0.5, linestyle='--')


        if len(All_kor_stock) < 31:
            plt.xticks([0, len(All_kor_stock)], labels=[self.S, self.E])
        elif 55 < len(All_kor_stock) < 75:
            plt.xticks([0, len(All_kor_stock)], labels=[self.S, self.E])
        else:
            plt.xticks([0, int(len(All_kor_stock) / 2), len(All_kor_stock)],
                       labels=[self.S, ''.join(self.predic12_1_set_SE.index[int(len(self.predic12_1_set_SE) / 2)]), self.E])
        plt.grid(True)
        plt.ylabel('Percent %')
        plt.show()
        
        
        #find mdd

            
            


In [4]:
        
# 대표적인 포트폴리오들과 예측 국면별 자산 최적 비중을 실행한 백테스팅 결과를 비교
class Strategy:

    def __init__(self):
        while (1):
            try:
                print("\n전략 실행은 2000-01 부터 가능합니다. YYYY-MM 양식을 지켜주세요")
                self.S = input("전략실행 시작 연-월을 입력하세요. 월초부터 실행합니다. ex)2003-03 : ")
                print("\n전략 실행은 2022-06 까지 가능합니다. YYYY-MM 양식을 지켜주세요")
                pre_E = input("전략실행 종료 연-월을 입력하세요. 월말까지 실행합니다. ex)2021-05 : ")
                self.E = ''.join(indexing_predic.index[(predic.index[predic.index[predic['date'] == pre_E]] + 1).tolist()])
            except:
                print('\n---------------------------------------------------')
                print(" 날짜입력 오류입니다. 1월인경우 01로 입력 바랍니다.")
                print('---------------------------------------------------\n')
                print("아무키나 입력하면 시작시간부터 다시 설정합니다.")
                self.Q = input("혹시 프로그램을 종료하고 싶다면 q를 입력해주세요.(대소문자 모두 가능) : ")
                print('\n\n\n')
                if self.Q == 'q' or self.Q == 'Q':
                    print('\n프로그램을 종료합니다. 감사합니다.')
                    exit()
                else:
                    continue
            

            self.predic12_1 = indexing_predic.loc[:, 'pred']
            self.predic12_1_set_SE = indexing_predic.loc[self.S : self.E]
              

            self.predic12_1_usa = indexing_predic_usa.loc[:, 'pred']
            self.predic12_1_set_SE_usa = indexing_predic_usa.loc[self.S : self.E]
     

            if self.predic12_1_set_SE.empty:
                print('\n---------------------------------------------------')
                print(" 날짜입력 오류입니다. 1월인경우 01로 입력 바랍니다.")
                print('---------------------------------------------------\n')

                continue


            else:
                break
                
        '''         
        while (1):
            print('미국과 한국의 포트폴리오 비율을 입력합니다. 입력값의 합이 10이 되게 해주세요 \n한국:미국 = 6:4 라면, 6입력  \n한국:미국 = 6:4 라면, 4입력')
            try:
                self.K = int(input("\n한국비율 입력. \n한국:미국 = 6:4 라면, 6입력 "))
                self.U = int(input("\n한국비율 입력. \n한국:미국 = 6:4 라면, 4입력 "))
      
                if (self.K + self.U) != 10:
                    print('\n한국 + 미국의 입력값이 총 10이 되게 해주세요\n')
                    continue
                else:
                    print('\n--------------------')
                    print("한국과 미국의 비율은 {} : {}".format(self.K, self.U))
                    print('--------------------\n')
                    break
            except:
                print('\n--------------------')
                print("숫자를 다시 입력해주세요")
                print('--------------------\n')
                continue

        '''    

        self.Collected_Data_fix_kor = pd.DataFrame()
        self.Collected_Data_fix_usa = pd.DataFrame()
        self.Collected_Data_fix_kor = indexing_bt.loc['{}'.format(self.S):'{}'.format(self.E)]
        self.Collected_Data_fix_usa = indexing_bt_usa.loc['{}'.format(self.S):'{}'.format(self.E)]

          
        
        
    # 고정된 포트폴리오를 계산하기 위한 함수 
    def Backtesting_manualset_kor(self, a, b, c, d, Data):

        stock = a
        bond = b
        gold = c
        commodity = d

        self.list_for_percent = []
        list_for_var = []

        R = None

        for i in range(0, len(Data)):
            gold = gold * (1 + Data['gold'][i])  
            bond = bond * (1 + Data['k10bond'][i])
            stock = stock * (1 + Data['kospi'][i])
            commodity = commodity * (1 + Data['commodity'][i])

            sum_for_var = gold + bond + stock + commodity
            self.list_for_percent.append(sum_for_var - 100)

            if R:  
                r = (sum_for_var - R) / R * 100
                list_for_var.append(r)
            R = sum_for_var

        rate_of_return = (gold + bond + stock + commodity - 100)  
        var_portfolio = np.array(list_for_var).var()  
        Volatility_standard_deviation = np.sqrt(var_portfolio)
        Sharpe_ratio = (rate_of_return / 100) / Volatility_standard_deviation  
        return_dataframe = pd.DataFrame(
            {'Return %': [round(rate_of_return, 4)], 'Sharpe': [round(Sharpe_ratio, 4)],
             'Volatility': [round(Volatility_standard_deviation, 4)]})

        print(
            "한국주식 {}%, 한국채권 10년 {}%, 금 {}%, 원자재 {}% 포트폴리오 의 결과 ".format(
                a, b, c, d))

        print('Return %: {}, Sharpe: {}, Volatility: {}\n'.format(round(return_dataframe['Return %'][0], 4),
                                                                  round(return_dataframe['Sharpe'][0], 4),
                                                                  round(return_dataframe['Volatility'][0], 4)))

        return self.list_for_percent


    # 전략 실행을 위한 백테스팅 함수
    def Backtesting_monthly_kor(self, a, b, c, d, Data, pre_sum, for_var):

        stock = pre_sum * a / 100
        bond = pre_sum * b / 100
        gold = pre_sum * c / 100
        commodity = pre_sum * d / 100

        list_for_mdd = []

        R = pre_sum
        for i in range(0, len(Data)):
            gold = gold * (1 + Data['gold'][i]) 
            bond = bond * (1 + Data['k10bond'][i])
            stock = stock * (1 + Data['kospi'][i])
            commodity = commodity * (1 + Data['commodity'][i])

            sum_for_mdd_var = gold + bond + stock + commodity
            list_for_mdd.append(sum_for_mdd_var)

            r = (sum_for_mdd_var - R) / R * 100
            for_var.append(r)

            R = sum_for_mdd_var

        self.df = pd.DataFrame(list_for_mdd)

        return self.df
    
    # 클래스 실행에 필요한 구간 데이터 모으는 함수
    def Select_Date_for_STG_kor(self, Start, End):  

        self.total = indexing_bt.loc['{}'.format(Start):'{}'.format(End)]

        return self.total
    def Select_Date_for_STG_usa(self, Start, End):  

        self.total = indexing_bt_usa.loc['{}'.format(Start):'{}'.format(End)]

        return self.total

    def Backtesting_manualset_usa(self, a, b, c, d, Data):

        stock = a
        bond = b
        gold = c
        commodity = d

        self.list_for_percent = []
        list_for_var = []

        R = None

        for i in range(0, len(Data)):
            gold = gold * (1 + Data['gold'][i])  
            bond = bond * (1 + Data['u10bond'][i])
            stock = stock * (1 + Data['snp500'][i])
            commodity = commodity * (1 + Data['commodity'][i])

            sum_for_var = gold + bond + stock + commodity
            self.list_for_percent.append(sum_for_var - 100)

            if R:  
                r = (sum_for_var - R) / R * 100
                list_for_var.append(r)
            R = sum_for_var

        rate_of_return = (gold + bond + stock + commodity - 100)  
        var_portfolio = np.array(list_for_var).var()  
        Volatility_standard_deviation = np.sqrt(var_portfolio)
        Sharpe_ratio = (rate_of_return / 100) / Volatility_standard_deviation  
        return_dataframe = pd.DataFrame(
            {'Return %': [round(rate_of_return, 4)], 'Sharpe': [round(Sharpe_ratio, 4)],
             'Volatility': [round(Volatility_standard_deviation, 4)]})

        print(
            "미국주식 {}%, 미국채권 10년 {}%, 금 {}%, 원자재 {}% 포트폴리오 의 결과 ".format(
                a, b, c, d))

        print('Return %: {}, Sharpe: {}, Volatility: {}\n'.format(round(return_dataframe['Return %'][0], 4),
                                                                  round(return_dataframe['Sharpe'][0], 4),
                                                                  round(return_dataframe['Volatility'][0], 4)))

        return self.list_for_percent


    # 전략 실행을 위한 백테스팅 함수
    def Backtesting_monthly_usa(self, a, b, c, d, Data, pre_sum, for_var):

        stock = pre_sum * a / 100
        bond = pre_sum * b / 100
        gold = pre_sum * c / 100
        commodity = pre_sum * d / 100

        list_for_mdd = []

        R = pre_sum
        for i in range(0, len(Data)):
            gold = gold * (1 + Data['gold'][i]) 
            bond = bond * (1 + Data['u10bond'][i])
            stock = stock * (1 + Data['snp500'][i])
            commodity = commodity * (1 + Data['commodity'][i])

            sum_for_mdd_var = gold + bond + stock + commodity
            list_for_mdd.append(sum_for_mdd_var)

            r = (sum_for_mdd_var - R) / R * 100
            for_var.append(r)

            R = sum_for_mdd_var

        self.df = pd.DataFrame(list_for_mdd)

        return self.df    
        
        
    
        
        
    # 각 포트폴리오들의 백테스팅 결과값과 그리프를 프린트합니다.        
    def Run(self):

        print('\n\n')
        kor_6_4 = pd.DataFrame(self.Backtesting_manualset_kor(60, 40, 0, 0, self.Collected_Data_fix_kor),
                                                columns=['6_4'])
        usa_6_4 = pd.DataFrame(self.Backtesting_manualset_usa(60, 40, 0, 0, self.Collected_Data_fix_usa),
                                                columns=['6_4'])
        
        
        All_kor_stock = pd.DataFrame(self.Backtesting_manualset_kor(100, 0, 0, 0, self.Collected_Data_fix_kor),
                                                 columns=['10_0'])
        
        All_usa_stock = pd.DataFrame(self.Backtesting_manualset_usa(100, 0, 0, 0, self.Collected_Data_fix_usa),
                                                 columns=['10_0'])

        #print('아래는 Reference인 All Weather Portfolio 입니다.')
        kor_Allweather_reference = pd.DataFrame(
            self.Backtesting_manualset_kor(30, 50, 10, 10, self.Collected_Data_fix_kor), columns=['reference '])
        usa_Allweather_reference = pd.DataFrame(
            self.Backtesting_manualset_usa(30, 50, 10, 10, self.Collected_Data_fix_usa), columns=['reference '])

        start = [100]
  
        array_for_STG_kor = pd.DataFrame(start)
        array_for_STG_usa = pd.DataFrame(start)
        array_for_STG_kor_var = []
        array_for_STG_usa_var = []
  

        print("Calculating kor strategies...")

        for i in self.predic12_1_set_SE.index:

   
            if i == self.predic12_1_set_SE.index[-1]:
                break

            else:
                if self.predic12_1.loc[i] == 0:


                    Colected_Data_Monthly_kor = self.Select_Date_for_STG_kor(i, ''.join(
                        self.predic12_1.index[(predic.index[predic.index[predic['date'] == i]] + 1).tolist()]))

                    array_for_STG_kor = pd.concat(
                        [array_for_STG_kor,
                         self.Backtesting_monthly_kor(10, 30, 50, 10, Colected_Data_Monthly_kor,
                                                         array_for_STG_kor.iloc[-1], array_for_STG_kor_var)],
                        ignore_index=True)




                elif self.predic12_1.loc[i] == 1:
  
                    Colected_Data_Monthly_kor = self.Select_Date_for_STG_kor(i, ''.join(
                        self.predic12_1.index[(predic.index[predic.index[predic['date'] == i]] + 1).tolist()]))
                    array_for_STG_kor = pd.concat(
                        [array_for_STG_kor,
                         self.Backtesting_monthly_kor(30, 50, 10, 10, Colected_Data_Monthly_kor,
                                                         array_for_STG_kor.iloc[-1], array_for_STG_kor_var)],
                        ignore_index=True)

                
                elif self.predic12_1.loc[i] == 2:

                    if (predic.index[predic.index[predic['date'] == i]] + 1).tolist()[0] < len(indexing_predic.index):
                        Colected_Data_Monthly_kor = self.Select_Date_for_STG_kor(i, ''.join(
                            self.predic12_1.index[(predic.index[predic.index[predic['date'] == i]] + 1).tolist()]))
                        array_for_STG_kor = pd.concat(
                            [array_for_STG_kor,
                             self.Backtesting_monthly_kor(24, 50, 16, 10, Colected_Data_Monthly_kor,
                                                             array_for_STG_kor.iloc[-1],
                                                             array_for_STG_kor_var)],
                            ignore_index=True)



                    else:
                        array_for_STG_kor = array_for_STG_kor.drop(index=0, axis=0)

                        continue

        
                elif self.predic12_1.loc[i] == 3:

                    Colected_Data_Monthly_kor = self.Select_Date_for_STG_kor(i, ''.join(
                        self.predic12_1.index[(predic.index[predic.index[predic['date'] == i]] + 1).tolist()]))
                    array_for_STG_kor = pd.concat(
                        [array_for_STG_kor,
                         self.Backtesting_monthly_kor(10, 30, 10, 50, Colected_Data_Monthly_kor,
                                                         array_for_STG_kor.iloc[-1], array_for_STG_kor_var)],
                        ignore_index=True)



                else:
                    print('Something.. Error')
                    
                    
        print("KOR_done...\n--------------------\n")
            
        
        
        print("Calculating usa strategies...")

        for i in self.predic12_1_set_SE_usa.index:
    
    
            if i == self.predic12_1_set_SE_usa.index[-1]:
                break
    
            else:
                if self.predic12_1_usa.loc[i] == 0:
    
                    Colected_Data_Monthly_usa = self.Select_Date_for_STG_usa(i, ''.join(
                        self.predic12_1_usa.index[(predic_usa.index[predic_usa.index[predic_usa['date'] == i]] + 1).tolist()]))
    
                    array_for_STG_usa = pd.concat(
                        [array_for_STG_usa,
                         self.Backtesting_monthly_usa(10, 50, 30, 10, Colected_Data_Monthly_usa,
                                                      array_for_STG_usa.iloc[-1], array_for_STG_usa_var)],
                        ignore_index=True)
    
    
    
    
                elif self.predic12_1_usa.loc[i] == 1:
    
                    Colected_Data_Monthly_usa = self.Select_Date_for_STG_usa(i, ''.join(
                        self.predic12_1_usa.index[(predic_usa.index[predic_usa.index[predic_usa['date'] == i]] + 1).tolist()]))
                    array_for_STG_usa = pd.concat(
                        [array_for_STG_usa,
                         self.Backtesting_monthly_usa(12, 50, 28, 10, Colected_Data_Monthly_usa,
                                                      array_for_STG_usa.iloc[-1], array_for_STG_usa_var)],
                        ignore_index=True)
    
    
                elif self.predic12_1_usa.loc[i] == 2:
    
                    if (predic_usa.index[predic_usa.index[predic_usa['date'] == i]] + 1).tolist()[0] < len(indexing_predic_usa.index):
                        Colected_Data_Monthly_usa = self.Select_Date_for_STG_usa(i, ''.join(
                            self.predic12_1_usa.index[(predic_usa.index[predic_usa.index[predic_usa['date'] == i]] + 1).tolist()]))
                        array_for_STG_usa = pd.concat(
                            [array_for_STG_usa,
                             self.Backtesting_monthly_usa(30, 50, 10, 10, Colected_Data_Monthly_usa,
                                                          array_for_STG_usa.iloc[-1],
                                                          array_for_STG_usa_var)],
                            ignore_index=True)
    
    
    
                    else:
                        array_for_STG_usa = array_for_STG_usa.drop(index=0, axis=0)
    
                        continue
    
    
                elif self.predic12_1_usa.loc[i] == 3:
    
                    Colected_Data_Monthly_usa = self.Select_Date_for_STG_usa(i, ''.join(
                        self.predic12_1_usa.index[(predic_usa.index[predic_usa.index[predic_usa['date'] == i]] + 1).tolist()]))
                    array_for_STG_usa = pd.concat(
                        [array_for_STG_usa,
                         self.Backtesting_monthly_usa(10, 50, 30, 10, Colected_Data_Monthly_usa,
                                                      array_for_STG_usa.iloc[-1], array_for_STG_usa_var)],
                        ignore_index=True)
    
    
    
                else:
                    print('Something.. Error')
            
        print("USA_done...\n\n")
       

        kor_var = np.array(array_for_STG_kor_var).var()
        kor_standard_deviation = np.sqrt(kor_var)
        array_for_STG_kor = array_for_STG_kor.drop([0])
        

        
        usa_var = np.array(array_for_STG_usa_var).var()
        usa_standard_deviation = np.sqrt(usa_var)
        array_for_STG_usa = array_for_STG_usa.drop([0])
        
        print("전략 kor 의 결과 ")
        print('Return %: {}, Sharpe: {}, Volatility: {}\n'.format(round((array_for_STG_kor.iloc[-1]) - 100, 4)[0],
                                                              round(((array_for_STG_kor.iloc[
                                                                          -1] - 100) / 100) / kor_standard_deviation,
                                                                    4)[0], round(kor_standard_deviation, 4)))  
        print("전략 usa 의 결과 ")
        print('Return %: {}, Sharpe: {}, Volatility: {}\n'.format(round((array_for_STG_usa.iloc[-1]) - 100, 4)[0],
                                                              round(((array_for_STG_usa.iloc[
                                                                          -1] - 100) / 100) / usa_standard_deviation,
                                                                    4)[0], round(usa_standard_deviation, 4)))    
        
        
        print("전략 KOR:USA = 1:9 의 결과 ")
        print('Sharpe: {}\n'.format(round(((array_for_STG_kor.iloc[-1] - 100) / 100) / kor_standard_deviation,4)[0]*0.1 + 0.9* round(((array_for_STG_usa.iloc[-1] - 100) / 100) / usa_standard_deviation,4)[0]
))    
        
        print("전략 KOR:USA = 1:9 의 결과 ")
        print('Sharpe: {}\n'.format(round(((array_for_STG_kor.iloc[-1] - 100) / 100) / kor_standard_deviation,4)[0]*0.1 + 0.9* round(((array_for_STG_usa.iloc[-1] - 100) / 100) / usa_standard_deviation,4)[0]
))    
        print("전략 KOR:USA = 2:8 의 결과 ")
        print('Sharpe: {}\n'.format(round(((array_for_STG_kor.iloc[-1] - 100) / 100) / kor_standard_deviation,4)[0]*0.2 + 0.8* round(((array_for_STG_usa.iloc[-1] - 100) / 100) / usa_standard_deviation,4)[0]
))    
        print("전략 KOR:USA = 3:7 의 결과 ")
        print('Sharpe: {}\n'.format(round(((array_for_STG_kor.iloc[-1] - 100) / 100) / kor_standard_deviation,4)[0]*0.3 + 0.7* round(((array_for_STG_usa.iloc[-1] - 100) / 100) / usa_standard_deviation,4)[0]
))    
        print("전략 KOR:USA = 4:6 의 결과 ")
        print('Sharpe: {}\n'.format(round(((array_for_STG_kor.iloc[-1] - 100) / 100) / kor_standard_deviation,4)[0]*0.4 + 0.6* round(((array_for_STG_usa.iloc[-1] - 100) / 100) / usa_standard_deviation,4)[0]
))    
        print("전략 KOR:USA = 5:5 의 결과 ")
        print('Sharpe: {}\n'.format(round(((array_for_STG_kor.iloc[-1] - 100) / 100) / kor_standard_deviation,4)[0]*0.5 + 0.5* round(((array_for_STG_usa.iloc[-1] - 100) / 100) / usa_standard_deviation,4)[0]
))    
        print("전략 KOR:USA = 6:4 의 결과 ")
        print('Sharpe: {}\n'.format(round(((array_for_STG_kor.iloc[-1] - 100) / 100) / kor_standard_deviation,4)[0]*0.6 + 0.4* round(((array_for_STG_usa.iloc[-1] - 100) / 100) / usa_standard_deviation,4)[0]
))    
        print("전략 KOR:USA = 7:3 의 결과 ")
        print('Sharpe: {}\n'.format(round(((array_for_STG_kor.iloc[-1] - 100) / 100) / kor_standard_deviation,4)[0]*0.7 + 0.3* round(((array_for_STG_usa.iloc[-1] - 100) / 100) / usa_standard_deviation,4)[0]
))    
        print("전략 KOR:USA = 8:2 의 결과 ")
        print('Sharpe: {}\n'.format(round(((array_for_STG_kor.iloc[-1] - 100) / 100) / kor_standard_deviation,4)[0]*0.8 + 0.2* round(((array_for_STG_usa.iloc[-1] - 100) / 100) / usa_standard_deviation,4)[0]
))    
        print("전략 KOR:USA = 9:1 의 결과 ")
        print('Sharpe: {}\n'.format(round(((array_for_STG_kor.iloc[-1] - 100) / 100) / kor_standard_deviation,4)[0]*0.9 + 0.1* round(((array_for_STG_usa.iloc[-1] - 100) / 100) / usa_standard_deviation,4)[0]
))    
  
        
        
        x = All_kor_stock.index  
        y = All_kor_stock.to_numpy()
        y1 = All_usa_stock.to_numpy()
        
        y111 = kor_6_4.to_numpy()
        y112 = usa_6_4.to_numpy()
        y113 = kor_Allweather_reference.to_numpy()
        y114 = usa_Allweather_reference.to_numpy()        
        y115 = y113*0.5 + y114*0.5 

        y4 = array_for_STG_kor.to_numpy() - 100 #kor
        y5 = array_for_STG_usa.to_numpy() - 100 #usa

        y6 = y4*0.1 + y5*0.9
        y7 = y4*0.2 + y5*0.8
        y8 = y4*0.3 + y5*0.7
        y9 = y4*0.4 + y5*0.6
        y10 = y4*0.5 + y5*0.5
        y11 = y4*0.6 + y5*0.4
        y12 = y4*0.7 + y5*0.3
        y13 = y4*0.8 + y5*0.2
        y14 = y4*0.9 + y5*0.1
   



        plt.figure(figsize=(40, 18))
        plt.plot(x, y, c='blue', alpha=0.2, label='KOR Stock 100%, Portfolio')
        plt.plot(x, y1, c='orange', alpha=0.2, label='USA Stock 100%, Portfolio')
        #plt.plot(x, y2, c='green', alpha=0.2, label='Stock 100% Portfolio')
        #plt.plot(x, y10, c='black', alpha=0.6, label='Ray Dalio Reference AWP')


        plt.plot(x, y4, c='purple', label='Our strategy_kor')
        plt.plot(x, y5, c='red', label='Our strategy_usa')
        
        plt.plot(x, y111, alpha=0.6, label='KOR Stock 60%, Bond 40% Portfolio')        
        plt.plot(x, y112, alpha=0.6,label='USA Stock 60%, Bond 40% Portfolio')
        plt.plot(x, y113, alpha=0.6,label='KOR Ray Dalio Reference AWP')        
        plt.plot(x, y114, alpha=0.6,label='USA Ray Dalio Reference AWP')        
        plt.plot(x, y115, alpha=0.6,label='KOR : USA = 1:1 Ray Dalio Reference AWP') 
        
        plt.plot(x, y6, alpha=0.4, label='Our strategy_KOR:USA = 1:9')
        plt.plot(x, y7, alpha=0.4,  label='Our strategy_KOR:USA = 2:8')
        plt.plot(x, y8, alpha=0.4,  label='Our strategy_KOR:USA = 3:7')
        plt.plot(x, y9, alpha=0.4,  label='Our strategy_KOR:USA = 4:6')
        plt.plot(x, y10,alpha=0.4,  label='Our strategy_KOR:USA = 5:5')
        plt.plot(x, y11,alpha=0.4,  label='Our strategy_KOR:USA = 6:4')
        plt.plot(x, y12,alpha=0.4,  label='Our strategy_KOR:USA = 7:3')
        plt.plot(x, y13,alpha=0.4,  label='Our strategy_KOR:USA = 8:2')
        plt.plot(x, y14, alpha=0.4, label='Our strategy_KOR:USA = 9:1')
        
        
 
        
        


        plt.legend(loc='upper left', frameon=False, fontsize=15)
        plt.axhline(y=0, color='black', linewidth=1, alpha=0.5, linestyle='--')


        if len(All_kor_stock) < 31:
            plt.xticks([0, len(All_kor_stock)], labels=[self.S, self.E])
        elif 55 < len(All_kor_stock) < 75:
            plt.xticks([0, len(All_kor_stock)], labels=[self.S, self.E])
        else:
            plt.xticks([0, int(len(All_kor_stock) / 2), len(All_kor_stock)],
                       labels=[self.S, ''.join(self.predic12_1_set_SE.index[int(len(self.predic12_1_set_SE) / 2)]), self.E])
        plt.grid(True)
        plt.ylabel('Percent %')
        plt.show()


In [ ]:
backtesting = Strategy().Run()


전략 실행은 2000-01 부터 가능합니다. YYYY-MM 양식을 지켜주세요
전략실행 시작 연-월을 입력하세요. 월초부터 실행합니다. ex)2003-03 : 2000-01

전략 실행은 2022-06 까지 가능합니다. YYYY-MM 양식을 지켜주세요
전략실행 종료 연-월을 입력하세요. 월말까지 실행합니다. ex)2021-05 : 2017-01



한국주식 60%, 한국채권 10년 40%, 금 0%, 원자재 0% 포트폴리오 의 결과 
Return %: 96.0773, Sharpe: 1.2023, Volatility: 0.7991

미국주식 60%, 미국채권 10년 40%, 금 0%, 원자재 0% 포트폴리오 의 결과 
Return %: 66.8784, Sharpe: 1.2417, Volatility: 0.5386

한국주식 100%, 한국채권 10년 0%, 금 0%, 원자재 0% 포트폴리오 의 결과 
Return %: 101.3094, Sharpe: 0.6942, Volatility: 1.4593

미국주식 100%, 미국채권 10년 0%, 금 0%, 원자재 0% 포트폴리오 의 결과 
Return %: 54.5963, Sharpe: 0.4653, Volatility: 1.1733

한국주식 30%, 한국채권 10년 50%, 금 10%, 원자재 10% 포트폴리오 의 결과 
Return %: 116.7862, Sharpe: 2.307, Volatility: 0.5062

미국주식 30%, 미국채권 10년 50%, 금 10%, 원자재 10% 포트폴리오 의 결과 
Return %: 101.3086, Sharpe: 2.3004, Volatility: 0.4404

Calculating kor strategies...


In [ ]:
MDD().Run()